In [4]:
import pandas as pd
import importlib
import sys
sys.path.append('../src')
import data
importlib.reload(data)
from data import save_data, load_data, manipulate_data, check_for_na, check_data_type, test_unit_dtype, test_unit_less_than_or_greater_than, test_unit_between, look_at_missing_values




In [2]:
df_USA_2018_2019_2020 = load_data('../data/raw/revised-seasonally-adjusted-indexes-2021.xlsx', '.xlsx')
df_Canada_2018_2019_2020 = load_data('../data/raw/18100004.csv', '.csv')
df_Canada_Sales = load_data('../data/raw/16100047.csv', '.csv')
df_USA_Sales = load_data('../data/raw/M3-mf.csv', '.csv')
df_Canada_Number_Groceries = load_data('../data/raw/20100056.csv', '.csv')

df_category_dict = load_data('../data/processed/dict_category_USA.xlsx', '.xlsx')
df_datatypes_dict = load_data('../data/processed/dict_datatypes_USA.csv', '.csv')
df_date_dict = load_data('../data/processed/dict_date_USA.csv', '.csv')







/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/data/make_dataset.py:41: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_filepath, header = header)
/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/data/make_dataset.py:41: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_filepath, header = header)


In [3]:
# Ensure these columns exist and contain numeric data
df_category_dict = df_category_dict[['cat_idx', 'cat_desc']]
df_datatypes_dict = df_datatypes_dict[['dt_idx', 'dt_desc']]
df_date_dict = df_date_dict[['per_idx', 'per_name']]  # Assuming 'per_desc' exists

# Convert to int64, ignoring any errors
df_category_dict['cat_idx'] = pd.to_numeric(df_category_dict['cat_idx'], errors='coerce').astype('int64')
df_datatypes_dict['dt_idx'] = pd.to_numeric(df_datatypes_dict['dt_idx'], errors='coerce').astype('int64')
df_date_dict['per_idx'] = pd.to_numeric(df_date_dict['per_idx'], errors='coerce').astype('int64')

print(df_category_dict['cat_idx'].dtype)


# Create mapping dictionaries
cat_dict = dict(zip(df_category_dict['cat_idx'], df_category_dict['cat_desc']))
dt_dict = dict(zip(df_datatypes_dict['dt_idx'], df_datatypes_dict['dt_desc']))
per_dict = dict(zip(df_date_dict['per_idx'], df_date_dict['per_name']))

# Apply mapping
df_USA_Sales['cat_idx'] = df_USA_Sales['cat_idx'].map(cat_dict)
df_USA_Sales['dt_idx'] = df_USA_Sales['dt_idx'].map(dt_dict)
df_USA_Sales['per_idx'] = df_USA_Sales['per_idx'].map(per_dict)



int64


In [4]:
df_USA_Sales['per_idx'] = df_USA_Sales['per_idx'].astype(str)
df_USA_Sales['Month'] = df_USA_Sales['per_idx'].str.split('-').str[0]
df_USA_Sales['Year'] = df_USA_Sales['per_idx'].str.split('-').str[1]
dict_year = {}
for key, value in zip(range(92, 100), range(1992, 2000)):
    dict_year[str(key)] = str(value)
for key, value in zip(range(2, 26), range(2000, 2026)):
    dict_year[str(key)] = str(value)

df_USA_Sales['Year'] = df_USA_Sales['Year'].map(dict_year)
df_USA_Sales['REF_DATE'] = pd.to_datetime(df_USA_Sales['Year'] + '-' + df_USA_Sales['Month'] + '-01')
df_USA_Sales.drop(columns = ['per_idx', 'Year', 'Month'], inplace=True)
print(df_USA_Sales.head())

for column in df_USA_Sales['cat_idx'].unique():
    print(column)

               cat_idx              dt_idx  geo_idx  is_adj       val  \
0  Total Manufacturing  Value of Shipments        1       0  209438.0   
1  Total Manufacturing  Value of Shipments        1       0  232679.0   
2  Total Manufacturing  Value of Shipments        1       0  249673.0   
3  Total Manufacturing  Value of Shipments        1       0  239666.0   
4  Total Manufacturing  Value of Shipments        1       0  243231.0   

    REF_DATE  
0 1992-01-01  
1 1992-02-01  
2 1992-03-01  
3 1992-04-01  
4 1992-05-01  
Total Manufacturing
Manufacturing Excluding Transportation
Manufacturing Excluding Defense
Manufacturing with Unfilled Orders
Durable Goods
Wood Products
Nonmetallic Mineral Products
Primary Metals
Iron and Steel Mills and Ferroalloy and Steel Product Manufacturing
Aluminum and Nonferrous Metal Products
Ferrous Metal Foundries
Fabricated Metal Products
Machinery
Farm Machinery and Equipment Manufacturing
Construction Machinery Manufacturing
Mining, Oil, and Gas Field

In [5]:
df_USA_2018_2019_2020.head(20)


,ITEM,TITLE,seriesid,DATA_TYPE,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2017,243.618,244.006,243.892,244.193,244.004,244.163,244.243,245.183,246.435,246.626,247.284,247.805
1,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2018,248.743,249.439,249.581,250.146,250.779,251.118,251.323,251.749,252.239,252.862,252.657,252.551
2,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2019,252.470,253.135,254.273,255.163,255.325,255.361,255.900,256.179,256.596,257.305,257.788,258.263
3,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2020,258.682,259.007,258.165,256.094,255.944,257.217,258.543,259.580,260.190,260.352,260.721,261.564
4,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2021,262.200,263.346,265.028,266.727,268.599,270.955,272.184,273.092,274.214,276.590,278.524,280.126
5,SA0,All items,CUSR0000SA0,SEASONAL FACTOR,2017,99.680,99.835,99.963,100.136,100.299,100.324,100.222,100.137,100.156,100.015,99.751,99.483
6,SA0,All items,CUSR0000SA0,SEASONAL FACTOR,2018,99.648,99.820,99.989,100.160,100.323,100.347,100.272,100.158,100.079,100.009,99.755,99.478
7,SA0,All items,CUSR0000SA0,SEASONAL FACTOR,2019,99.700,99.858,99.972,100.151,100.300,100.306,100.262,100.148,100.064,100.016,99.775,99.501
8,SA0,All items,CUSR0000SA0,SEASONAL FACTOR,2020,99.725,99.873,99.981,100.115,100.176,100.225,100.216,100.130,100.035,100.014,99.811,99.583
9,SA0,All items,CUSR0000SA0,SEASONAL FACTOR,2021,99.764,99.874,99.943,100.123,100.222,100.273,100.301,100.174,100.035,100.000,99.793,99.527


In [6]:
df_USA_2018_2019_2020.sample(20)

,ITEM,TITLE,seriesid,DATA_TYPE,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
4683,SEEE04,"Telephone hardware, calculators, and other con...",CWSR0000SEEE04,SEASONALLY ADJUSTED INDEX,2020,16.196,16.097,15.950,15.722,15.888,15.413,15.187,14.742,14.750,14.706,14.001,13.621
2680,SERE,Other recreational goods,CUSR0000SERE,UNADJUSTED INDEX,2017,42.354,42.459,41.867,41.379,41.091,40.752,40.073,39.831,39.793,39.902,39.263,38.776
4377,SEAB,Boys' apparel,CWSR0000SEAB,UNADJUSTED INDEX,2019,111.094,112.630,106.571,104.587,104.479,103.399,101.073,104.580,112.589,111.463,106.006,103.540
5610,SEHG01,Water and sewerage maintenance,CWSR0000SEHG01,SEASONALLY ADJUSTED INDEX,2017,514.571,516.261,517.659,518.957,520.570,521.609,522.657,523.982,525.364,526.107,527.987,529.374
2544,SERA01,Televisions,CUSR0000SERA01,SEASONAL FACTOR,2021,98.563,99.200,98.924,98.986,100.091,100.894,101.532,102.331,102.158,101.004,99.218,97.266
3094,SS18041,Salt and other seasonings and spices,CUSR0000SS18041,SEASONALLY ADJUSTED INDEX,2021,159.597,159.851,159.688,160.713,160.845,159.962,161.149,160.281,163.188,164.918,169.295,169.192
1681,SEFJ03,Ice cream and related products,CUSR0000SEFJ03,SEASONALLY ADJUSTED INDEX,2018,221.673,217.551,220.143,219.496,218.980,221.191,220.582,221.427,221.394,221.465,220.834,221.800
1531,SEFD,Pork,CUSR0000SEFD,SEASONALLY ADJUSTED INDEX,2018,213.933,213.082,213.541,211.786,210.763,208.319,207.510,209.120,208.531,206.386,209.734,209.885
527,SAF1131,Fresh fruits and vegetables,CUSR0000SAF1131,SEASONALLY ADJUSTED INDEX,2019,352.051,353.826,356.566,352.403,349.744,347.976,349.126,348.315,346.167,348.036,346.392,344.285
3488,SAA1,Men's and boys' apparel,CWSR0000SAA1,SEASONAL FACTOR,2020,99.814,103.000,101.660,100.093,100.131,98.433,97.892,99.627,101.053,101.327,99.547,97.155


In [7]:
df_USA_2018_2019_2020.tail(20)

,ITEM,TITLE,seriesid,DATA_TYPE,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
6205,SS5702,Inpatient hospital services,CWSR0000SS5702,UNADJUSTED INDEX,2017,305.219,308.941,308.831,310.785,310.247,312.017,312.670,313.180,313.163,314.996,315.970,316.021
6206,SS5702,Inpatient hospital services,CWSR0000SS5702,UNADJUSTED INDEX,2018,321.465,322.262,322.726,323.116,323.420,324.752,324.882,323.736,324.249,323.871,323.779,324.164
6207,SS5702,Inpatient hospital services,CWSR0000SS5702,UNADJUSTED INDEX,2019,325.369,325.846,326.503,324.692,326.193,326.156,327.573,329.449,329.280,334.033,334.975,335.168
6208,SS5702,Inpatient hospital services,CWSR0000SS5702,UNADJUSTED INDEX,2020,339.147,338.947,339.856,341.061,341.915,342.395,342.807,341.980,342.974,341.580,342.460,342.190
6209,SS5702,Inpatient hospital services,CWSR0000SS5702,UNADJUSTED INDEX,2021,345.318,345.989,347.197,348.164,348.169,347.843,350.464,353.951,354.026,356.385,355.412,354.945
6210,SS5703,Outpatient hospital services,CWSR0000SS5703,SEASONALLY ADJUSTED INDEX,2017,689.826,698.595,700.369,708.572,709.995,714.165,716.433,716.148,718.641,719.273,718.899,721.774
6211,SS5703,Outpatient hospital services,CWSR0000SS5703,SEASONALLY ADJUSTED INDEX,2018,728.207,728.443,732.659,734.546,737.430,742.391,744.176,740.043,738.945,738.186,745.368,748.627
6212,SS5703,Outpatient hospital services,CWSR0000SS5703,SEASONALLY ADJUSTED INDEX,2019,743.415,742.990,743.529,740.201,743.387,741.714,746.458,753.931,753.861,759.609,761.366,763.790
6213,SS5703,Outpatient hospital services,CWSR0000SS5703,SEASONALLY ADJUSTED INDEX,2020,767.627,769.819,770.524,772.521,771.059,773.328,774.441,774.979,780.250,775.366,779.219,781.946
6214,SS5703,Outpatient hospital services,CWSR0000SS5703,SEASONALLY ADJUSTED INDEX,2021,783.978,784.166,788.346,788.559,791.285,794.723,796.905,799.357,802.159,804.601,803.050,803.774


In [8]:
for column in df_USA_2018_2019_2020.columns:
    print(check_data_type(df_USA_2018_2019_2020, column), sep = '\n')
    print("Is there any missing NA values for US dataset: ", check_for_na(df_USA_2018_2019_2020, column))
    

ITEM data type is: object
Is there any missing NA values for US dataset:  False
TITLE data type is: object
Is there any missing NA values for US dataset:  False
seriesid data type is: object
Is there any missing NA values for US dataset:  False
DATA_TYPE data type is: object
Is there any missing NA values for US dataset:  False
YEAR data type is: int64
Is there any missing NA values for US dataset:  False
JAN data type is: float64
Is there any missing NA values for US dataset:  False
FEB data type is: float64
Is there any missing NA values for US dataset:  False
MAR data type is: float64
Is there any missing NA values for US dataset:  False
APR data type is: float64
Is there any missing NA values for US dataset:  False
MAY data type is: float64
Is there any missing NA values for US dataset:  False
JUN data type is: float64
Is there any missing NA values for US dataset:  False
JUL data type is: float64
Is there any missing NA values for US dataset:  False
AUG data type is: float64
Is th

In [9]:
#there are no missing values in the US dataset and all CPI values are float values while YEAR is an integer value that will be converted to datetime format in pandas

In [10]:
for column in df_USA_2018_2019_2020.columns:
    if df_USA_2018_2019_2020[column].dtype == float:
        print(column, " is greater than 0: ", test_unit_less_than_or_greater_than(df_USA_2018_2019_2020, column, 0, "greater_than"))
        print(column, " is less than 1000: ", test_unit_less_than_or_greater_than(df_USA_2018_2019_2020, column, 1000, "less_than"))



JAN  is greater than 0:  True
JAN  is less than 1000:  True
FEB  is greater than 0:  True
FEB  is less than 1000:  True
MAR  is greater than 0:  True
MAR  is less than 1000:  True
APR  is greater than 0:  True
APR  is less than 1000:  True
MAY  is greater than 0:  True
MAY  is less than 1000:  True
JUN  is greater than 0:  True
JUN  is less than 1000:  True
JUL  is greater than 0:  True
JUL  is less than 1000:  True
AUG  is greater than 0:  True
AUG  is less than 1000:  True
SEP  is greater than 0:  True
SEP  is less than 1000:  True
OCT  is greater than 0:  True
OCT  is less than 1000:  True
NOV  is greater than 0:  True
NOV  is less than 1000:  True
DEC  is greater than 0:  True
DEC  is less than 1000:  True


In [11]:
df_USA_2018_2019_2020[(df_USA_2018_2019_2020['TITLE'] == 'All items') & (df_USA_2018_2019_2020['YEAR'] == 2018)].head(20)

,ITEM,TITLE,seriesid,DATA_TYPE,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
1,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2018,248.743,249.439,249.581,250.146,250.779,251.118,251.323,251.749,252.239,252.862,252.657,252.551
6,SA0,All items,CUSR0000SA0,SEASONAL FACTOR,2018,99.648,99.820,99.989,100.160,100.323,100.347,100.272,100.158,100.079,100.009,99.755,99.478
11,SA0,All items,CUSR0000SA0,UNADJUSTED INDEX,2018,247.867,248.991,249.554,250.546,251.588,251.989,252.006,252.146,252.439,252.885,252.038,251.233
3346,SA0,All items,CWSR0000SA0,SEASONALLY ADJUSTED INDEX,2018,242.911,243.595,243.575,244.142,244.848,245.206,245.343,245.846,246.332,247.007,246.634,246.199
3351,SA0,All items,CWSR0000SA0,SEASONAL FACTOR,2018,99.592,99.751,99.954,100.190,100.377,100.404,100.331,100.199,100.095,100.013,99.716,99.426
3356,SA0,All items,CWSR0000SA0,UNADJUSTED INDEX,2018,241.919,242.988,243.463,244.607,245.770,246.196,246.155,246.336,246.565,247.038,245.933,244.786


In [12]:
#note that there are two sesasonally adjusted indexes for All items for the year 2018. The first one is adjusted the primary time it was looked at - the second one is adjusted 5 years later. Anywhere there is two values we will take the mean of the two values to get the average seasonally adjusted index for the year 2018

#Not all the items in the US dataset are seasonally adjusted twice which is why we take the average.

In [13]:
df_USA_CPI_Formatted = manipulate_data(df_USA_2018_2019_2020, 'USA')

/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/data/make_dataset.py:137: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_USA_Single_Columns_for_2017_2018_2019_2020['REF_DATE'] = pd.to_datetime(df_USA_Single_Columns_for_2017_2018_2019_2020['REF_DATE'])


In [14]:
df_USA_CPI_Formatted.head()
print(df_USA_CPI_Formatted.tail())

    Airline fares  Alcoholic beverages  Alcoholic beverages at home  \
43        205.545             260.8720                     214.7530   
44        204.960             260.4105                     214.1665   
45        213.578             261.4845                     215.3540   
46        217.057             262.9495                     216.4955   
47        210.732             262.4410                     216.0255   

    All items  All items less energy  All items less food  \
43   256.3910               264.6895             254.2695   
44   257.0485               265.1635             255.0175   
45   257.1905               265.3655             255.1085   
46   257.5485               265.7715             255.5245   
47   258.4135               266.0000             256.3850   

    All items less food and energy  All items less food and shelter  \
43                        264.5960                          228.213   
44                        265.1390                          229.

In [15]:
for column in df_USA_CPI_Formatted.columns:
    print(column)

Airline fares
Alcoholic beverages
Alcoholic beverages at home
All items
All items less energy
All items less food
All items less food and energy
All items less food and shelter
All items less food, shelter, and energy
All items less food, shelter, energy, and used cars and trucks
All items less medical care
All items less shelter
Apparel
Apparel less footwear
Apples
Appliances
Bacon and related products
Bacon, breakfast sausage, and related products
Beef and veal
Beer, ale, and other malt beverages at home
Beverage materials including coffee and tea
Boys' and girls' footwear
Boys' apparel
Breakfast sausage and related products
Butter
Butter and margarine
Cable and satellite television service
Canned fruits
Canned fruits and vegetables
Canned vegetables
Car and truck rental
Carbonated drinks
Cereals and bakery products
Cereals and cereal products
Citrus fruits
Coffee
College tuition and fees
Commodities
Commodities less food
Commodities less food and beverages
Commodities less food and 

In [16]:
#see how Education has risen in 2018 and 2019 since 1984 which is the base value of 100
print("Education in the year 2018-2019", " is greater than 0: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Education", 0, "greater_than"))
print("Education in the year 2018-2019", " is greater than 150: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Education", 150, "greater_than"))
print("Education in the year 2018-2019", " is greater than 200: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Education", 200, "greater_than"))
print("Education in the year 2018-2019", " is greater than 250: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Education", 250, "greater_than"))
print("Education in the year 2018-2019", " is greater than 300: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Education", 300, "greater_than"))
print("Education in the year 2018-2019", " is less than 500: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Education", 500, "less_than"))
print("Education in the year 2018-2019", " is less than 800: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Education", 800, "less_than"))

print(df_USA_CPI_Formatted["Education"].describe())

Education in the year 2018-2019  is greater than 0:  True
Education in the year 2018-2019  is greater than 150:  True
Education in the year 2018-2019  is greater than 200:  True
Education in the year 2018-2019  is greater than 250:  True
Education in the year 2018-2019  is greater than 300:  False
Education in the year 2018-2019  is less than 500:  True
Education in the year 2018-2019  is less than 800:  True
count     48.000000
mean     260.064833
std        6.809547
min      249.141500
25%      253.938500
50%      260.385000
75%      266.115750
max      269.745000
Name: Education, dtype: float64


In [17]:
#see how Energy has risen in 2018 and 2019 since 1984 which is the base value of 100
print("Energy in the year 2018-2019", " is greater than 0: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Energy", 0, "greater_than"))
print("Energy in the year 2018-2019", " is greater than 150: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Energy", 150, "greater_than"))
print("Energy in the year 2018-2019", " is greater than 200: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Energy", 200, "greater_than"))
print("Energy in the year 2018-2019", " is greater than 250: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Energy", 250, "greater_than"))
print("Energy in the year 2018-2019", " is less than 500: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Energy", 500, "less_than"))
print("Energy in the year 2018-2019", " is less than 800: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "Energy", 800, "less_than"))

print(df_USA_CPI_Formatted["Energy"].describe())

Energy in the year 2018-2019  is greater than 0:  True
Energy in the year 2018-2019  is greater than 150:  True
Energy in the year 2018-2019  is greater than 200:  True
Energy in the year 2018-2019  is greater than 250:  False
Energy in the year 2018-2019  is less than 500:  True
Energy in the year 2018-2019  is less than 800:  True
count     48.000000
mean     208.832552
std       11.581002
min      178.364000
25%      201.796875
50%      212.847500
75%      217.681375
max      226.467500
Name: Energy, dtype: float64


In [18]:
#see how All-items has risen in 2018 and 2019 since 1984 which is the base value of 100
print("All items in the year 2018-2019", " is greater than 0: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "All items", 0, "greater_than"))
print("All items in the year 2018-2019", " is greater than 150: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "All items", 150, "greater_than"))
print("All items in the year 2018-2019", " is greater than 200: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "All items", 200, "greater_than"))
print("All items in the year 2018-2019", " is greater than 250: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "All items", 250, "greater_than"))
print("All items in the year 2018-2019", " is less than 500: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "All items", 500, "less_than"))
print("All items in the year 2018-2019", " is less than 800: ", test_unit_less_than_or_greater_than(df_USA_CPI_Formatted, "All items", 800, "less_than"))
print(df_USA_CPI_Formatted["All items"].describe())

All items in the year 2018-2019  is greater than 0:  True
All items in the year 2018-2019  is greater than 150:  True
All items in the year 2018-2019  is greater than 200:  True
All items in the year 2018-2019  is greater than 250:  True
All items in the year 2018-2019  is less than 500:  True
All items in the year 2018-2019  is less than 800:  True
count     48.000000
mean     249.547740
std        5.350473
min      240.666500
25%      245.583750
50%      249.784250
75%      253.882125
max      258.413500
Name: All items, dtype: float64


In [19]:
df_Canada_2018_2019_2020.head()


,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1914-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,NaN,1
1,1914-01,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,t,1
2,1914-01,Canada,2016A000011124,Goods and services,2002=100,17,units,0,v41691221,2.273,6.0,NaN,NaN,t,1
3,1914-02,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,NaN,1
4,1914-02,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,t,1


In [20]:
df_Canada_2018_2019_2020.tail(10)

,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
1112803,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Private transportation excluding gasoline,2002=100,17,units,0,v43972175,30.312,157.3,NaN,NaN,NaN,1
1112804,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Food and energy,2002=100,17,units,0,v41692840,30.286,205.7,NaN,NaN,NaN,1
1112805,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Fresh fruit and vegetables,2002=100,17,units,0,v41692838,30.294,219.2,NaN,NaN,NaN,1
1112806,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Energy,2002=100,17,units,0,v41692842,30.288,241.7,NaN,NaN,NaN,1
1112807,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Goods,2002=100,17,units,0,v41692830,30.274,156.8,NaN,NaN,NaN,1
1112808,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Durable goods,2002=100,17,units,0,v41692831,30.275,114.1,NaN,NaN,NaN,1
1112809,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Semi-durable goods,2002=100,17,units,0,v41692832,30.276,108.1,NaN,NaN,NaN,1
1112810,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Non-durable goods,2002=100,17,units,0,v41692833,30.277,195.6,NaN,NaN,NaN,1
1112811,2024-12,"Yellowknife, Northwest Territories",2011A00056106023,Services,2002=100,17,units,0,v41692834,30.282,161.5,NaN,NaN,NaN,1
1112812,2024-12,"Iqaluit, Nunavut",2011A00056204003,All-items,2002=100,17,units,0,v41713432,31.200,144.1,NaN,NaN,NaN,1


In [21]:
for column in df_Canada_2018_2019_2020.columns:
    print(check_data_type(df_Canada_2018_2019_2020, column), sep = '\n')
    print("Is there any missing NA values: ", check_for_na(df_Canada_2018_2019_2020, column))

REF_DATE data type is: object
Is there any missing NA values:  False
GEO data type is: object
Is there any missing NA values:  False
DGUID data type is: object
Is there any missing NA values:  True
Products and product groups data type is: object
Is there any missing NA values:  False
UOM data type is: object
Is there any missing NA values:  False
UOM_ID data type is: int64
Is there any missing NA values:  False
SCALAR_FACTOR data type is: object
Is there any missing NA values:  False
SCALAR_ID data type is: int64
Is there any missing NA values:  False
VECTOR data type is: object
Is there any missing NA values:  False
COORDINATE data type is: float64
Is there any missing NA values:  False
VALUE data type is: float64
Is there any missing NA values:  False
STATUS data type is: object
Is there any missing NA values:  True
SYMBOL data type is: float64
Is there any missing NA values:  True
TERMINATED data type is: object
Is there any missing NA values:  True
DECIMALS data type is: int64
Is 

In [22]:
#There seems to be missing values in DGUID, STATUS, SYMBOL, and TERMINATED but we are not using those columns and dropping them when we manipulate the data

In [23]:
#UOM is an object that has multiple unique values due to the CPI index being relative to the base year - we chose the year of 2002 as the base year for the Canadian dataset

df_Canada_CPI_Formatted = manipulate_data(df_Canada_2018_2019_2020, 'Canada')

UOM has two unique values in the dataset:  ['2002=100' '2013=100' '2017=100' '200212=100' '201104=100' '200704=100'
 '1992=100' '201812=100']
UOM now only has 2002 value:  ['2002=100']


In [24]:
#Our dataset now has 3 columns for the years 2017, 2018, and 2019 with Products and product groups arranged in order. The base year is 2002.
df_Canada_CPI_Formatted.head(50)

,Products and product groups,REF_DATE,VALUE
0,Air transportation,2017-01-01,134.800000
1,Air transportation,2017-02-01,135.200000
2,Air transportation,2017-03-01,133.700000
3,Air transportation,2017-04-01,134.700000
4,Air transportation,2017-05-01,139.400000
5,Air transportation,2017-06-01,140.300000
6,Air transportation,2017-07-01,144.200000
7,Air transportation,2017-08-01,143.000000
8,Air transportation,2017-09-01,140.400000
9,Air transportation,2017-10-01,144.300000


In [25]:
for column in df_Canada_Sales.columns:
    print(check_data_type(df_Canada_Sales, column), sep = '\n')
    print("Is there any missing NA values for US dataset: ", check_for_na(df_Canada_Sales, column))

REF_DATE data type is: object
Is there any missing NA values for US dataset:  False
GEO data type is: object
Is there any missing NA values for US dataset:  False
DGUID data type is: object
Is there any missing NA values for US dataset:  False
Principal statistics data type is: object
Is there any missing NA values for US dataset:  False
Seasonal adjustment data type is: object
Is there any missing NA values for US dataset:  False
North American Industry Classification System (NAICS) data type is: object
Is there any missing NA values for US dataset:  False
UOM data type is: object
Is there any missing NA values for US dataset:  False
UOM_ID data type is: int64
Is there any missing NA values for US dataset:  False
SCALAR_FACTOR data type is: object
Is there any missing NA values for US dataset:  False
SCALAR_ID data type is: int64
Is there any missing NA values for US dataset:  False
VECTOR data type is: object
Is there any missing NA values for US dataset:  False
COORDINATE data type 

In [26]:
#there are missing values in VALUE, lets format the data first and then look at the missing values

missing_value = look_at_missing_values(df_Canada_Sales, 'VALUE')

Missing data:          REF_DATE     GEO           DGUID  \
102      1992-01  Canada  2016A000011124   
103      1992-01  Canada  2016A000011124   
104      1992-01  Canada  2016A000011124   
105      1992-01  Canada  2016A000011124   
106      1992-01  Canada  2016A000011124   
...          ...     ...             ...   
1039715  2024-12  Canada  2016A000011124   
1039720  2024-12  Canada  2016A000011124   
1039721  2024-12  Canada  2016A000011124   
1039722  2024-12  Canada  2016A000011124   
1039723  2024-12  Canada  2016A000011124   

                                      Principal statistics  \
102                Sales of goods manufactured (shipments)   
103                Sales of goods manufactured (shipments)   
104                Sales of goods manufactured (shipments)   
105                Sales of goods manufactured (shipments)   
106                Sales of goods manufactured (shipments)   
...                                                    ...   
1039715  Total invento

In [27]:



df_Canada_Sales_Formatted = manipulate_data(df_Canada_Sales, 'Canada')

UOM has two unique values in the dataset:  ['Dollars' 'Ratio']
UOM now only has 2002 value:  ['Dollars']


In [28]:
df_Canada_Sales_Formatted.head(50)

,Principal statistics,North American Industry Classification System (NAICS),REF_DATE,VALUE
791132,Sales of goods manufactured (shipments),Manufacturing [31-33],2017-01-01,54032136.0
791133,Sales of goods manufactured (shipments),Non-durable goods industries,2017-01-01,25334426.0
791134,Sales of goods manufactured (shipments),Food manufacturing [311],2017-01-01,8249500.0
791135,Sales of goods manufactured (shipments),Chocolate and chocolate confectionery manufact...,2017-01-01,NaN
791136,Sales of goods manufactured (shipments),"Flour mixes, dough, and pasta manufacturing fr...",2017-01-01,NaN
791137,Sales of goods manufactured (shipments),Beverage and tobacco product manufacturing [312],2017-01-01,1163819.0
791138,Sales of goods manufactured (shipments),Textile mills [313],2017-01-01,139835.0
791139,Sales of goods manufactured (shipments),Textile product mills [314],2017-01-01,136539.0
791140,Sales of goods manufactured (shipments),Apparel manufacturing [315],2017-01-01,185875.0
791141,Sales of goods manufactured (shipments),"Women's, girls' and infants' cut and sew cloth...",2017-01-01,NaN


In [29]:
missing_value = look_at_missing_values(df_Canada_Sales_Formatted, 'VALUE')

Missing data:                                       Principal statistics  \
791135            Sales of goods manufactured (shipments)   
791136            Sales of goods manufactured (shipments)   
791141            Sales of goods manufactured (shipments)   
791142            Sales of goods manufactured (shipments)   
791143            Sales of goods manufactured (shipments)   
...                                                   ...   
888681  Goods or work in process, estimated values at ...   
889037  Finished goods manufactured, estimated values ...   
889042  Finished goods manufactured, estimated values ...   
889402  Total inventory, estimated values of total inv...   
889403  Total inventory, estimated values of total inv...   

       North American Industry Classification System (NAICS)   REF_DATE  VALUE  
791135  Chocolate and chocolate confectionery manufact...    2017-01-01    NaN  
791136  Flour mixes, dough, and pasta manufacturing fr...    2017-01-01    NaN  
791141  W

In [30]:
for unique_principal_statistics in df_Canada_Sales_Formatted['Principal statistics'].unique():
    for unique_item in df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'].unique():
        # Create a mask for the current group
        mask = (
            (df_Canada_Sales_Formatted['Principal statistics'] == unique_principal_statistics) &
            (df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'] == unique_item)
            &
            (df_Canada_Sales_Formatted['VALUE'].isnull())
        )
        
        # Print the missing values for the masked group
        print(
            f"Missing values for {unique_principal_statistics} and {unique_item}:",
            df_Canada_Sales_Formatted[mask]['REF_DATE']
        )

Missing values for Sales of goods manufactured (shipments) and Manufacturing [31-33]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Sales of goods manufactured (shipments) and Non-durable goods industries: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Sales of goods manufactured (shipments) and Food manufacturing [311]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Sales of goods manufactured (shipments) and Chocolate and chocolate confectionery manufacturing [31135]: 791135   2017-01-01
793776   2017-02-01
796417   2017-03-01
799058   2017-04-01
801699   2017-05-01
804340   2017-06-01
806981   2017-07-01
809622   2017-08-01
812263   2017-09-01
814904   2017-10-01
817545   2017-11-01
820186   2017-12-01
Name: REF_DATE, dtype: datetime64[ns]
Missing values for Sales of goods manufactured (shipments) and Flour mixes, dough, and pasta manufacturing from purchased flour [311824]: 791136   2017-01-01
793777   2017-02-01
79

In [31]:
# Ensure DataFrame is sorted appropriately
df_Canada_Sales_Formatted = df_Canada_Sales_Formatted.sort_values(by=['Principal statistics', 'North American Industry Classification System (NAICS)', 'REF_DATE'])


# Iterate through unique combinations of 'Principal statistics' and 'NAICS'
for unique_principal_statistics in df_Canada_Sales_Formatted['Principal statistics'].unique():
    for unique_item in df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'].unique():
        # Create a mask for the current group
        mask = (
            (df_Canada_Sales_Formatted['Principal statistics'] == unique_principal_statistics) &
            (df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'] == unique_item)
        )
        
        # Apply interpolation on the 'VALUE' column for the masked group
        df_Canada_Sales_Formatted.loc[mask, 'VALUE'] = (
            df_Canada_Sales_Formatted.loc[mask, 'VALUE']
            .interpolate(method='linear', limit_direction='both')
        )




In [32]:
missing_value = look_at_missing_values(df_Canada_Sales_Formatted, 'VALUE')



Missing data:                                       Principal statistics  \
792932  Finished goods manufactured, estimated values ...   
795573  Finished goods manufactured, estimated values ...   
798214  Finished goods manufactured, estimated values ...   
800855  Finished goods manufactured, estimated values ...   
803496  Finished goods manufactured, estimated values ...   
...                                                   ...   
810345  Unfilled orders, estimated values of orders at...   
812986  Unfilled orders, estimated values of orders at...   
815627  Unfilled orders, estimated values of orders at...   
818268  Unfilled orders, estimated values of orders at...   
820909  Unfilled orders, estimated values of orders at...   

       North American Industry Classification System (NAICS)   REF_DATE  VALUE  
792932  Chocolate and chocolate confectionery manufact...    2017-01-01    NaN  
795573  Chocolate and chocolate confectionery manufact...    2017-02-01    NaN  
798214  C

In [33]:
#There are still missing values despite interpolation, lets look closer at those missing values to see if we need to drop them

for unique_principal_statistics in df_Canada_Sales_Formatted['Principal statistics'].unique():
    for unique_item in df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'].unique():
        # Create a mask for the current group
        mask = (
            (df_Canada_Sales_Formatted['Principal statistics'] == unique_principal_statistics) &
            (df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'] == unique_item)
            &
            (df_Canada_Sales_Formatted['VALUE'].isnull())
        )
        
        # Print the missing values for the masked group
        print(
            f"Missing values for {unique_principal_statistics} and {unique_item}:",
            df_Canada_Sales_Formatted[mask]['REF_DATE']
        )

Missing values for Finished goods manufactured, estimated values at end of  month and Aerospace product and parts manufacturing [3364]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Apparel manufacturing [315]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Beverage and tobacco product manufacturing [312]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Chemical manufacturing [325]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Chocolate and chocolate confectionery manufacturing [31135]: 792932   2017-01-01
795573   2017-02-01
798214   2017-03-01
800855   2017-04-01
803496   2017-05-01
806137   2017-06-01
808778   2017-07-01
811419   2

In [34]:
#For the missing values that are greater than 30 items we will drop them since we cannot use the data if it does not span from 2017-2020. The rest of the data we will leave to analysis to see if they want to use a model to predict the missing data - we will not be using all the data so an ARIMA model or polynomial to predict the beginning data may be warranted if we use that sales data

for unique_principal_statistics in df_Canada_Sales_Formatted['Principal statistics'].unique():
    for unique_item in df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'].unique():
        # Create a mask for the current group
        mask = (
            (df_Canada_Sales_Formatted['Principal statistics'] == unique_principal_statistics) &
            (df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'] == unique_item)
            &
            (df_Canada_Sales_Formatted['VALUE'].isnull())
        )
        mask2 = (
            (df_Canada_Sales_Formatted['Principal statistics'] == unique_principal_statistics) &
            (df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'] == unique_item)
        )
        # Print the missing values for the masked group
        print(
            f"Missing values for {unique_principal_statistics} and {unique_item}:",
            df_Canada_Sales_Formatted[mask]['REF_DATE']
        )
        if len(df_Canada_Sales_Formatted[mask]['REF_DATE']) > 30:
            df_Canada_Sales_Formatted = df_Canada_Sales_Formatted.drop(df_Canada_Sales_Formatted[mask].index)

Missing values for Finished goods manufactured, estimated values at end of  month and Aerospace product and parts manufacturing [3364]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Apparel manufacturing [315]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Beverage and tobacco product manufacturing [312]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Chemical manufacturing [325]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Chocolate and chocolate confectionery manufacturing [31135]: 792932   2017-01-01
795573   2017-02-01
798214   2017-03-01
800855   2017-04-01
803496   2017-05-01
806137   2017-06-01
808778   2017-07-01
811419   2

In [35]:
for unique_principal_statistics in df_Canada_Sales_Formatted['Principal statistics'].unique():
    for unique_item in df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'].unique():
        # Create a mask for the current group
        mask = (
            (df_Canada_Sales_Formatted['Principal statistics'] == unique_principal_statistics) &
            (df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'] == unique_item)
            &
            (df_Canada_Sales_Formatted['VALUE'].isnull())
        )
        mask2 = (
            (df_Canada_Sales_Formatted['Principal statistics'] == unique_principal_statistics) &
            (df_Canada_Sales_Formatted['North American Industry Classification System (NAICS)'] == unique_item)
        )
        # Print the missing values for the masked group
        print(
            f"Missing values for {unique_principal_statistics} and {unique_item}:",
            df_Canada_Sales_Formatted[mask]['REF_DATE']
        )

Missing values for Finished goods manufactured, estimated values at end of  month and Aerospace product and parts manufacturing [3364]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Apparel manufacturing [315]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Beverage and tobacco product manufacturing [312]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Chemical manufacturing [325]: Series([], Name: REF_DATE, dtype: datetime64[ns])
Missing values for Finished goods manufactured, estimated values at end of  month and Chocolate and chocolate confectionery manufacturing [31135]: 792932   2017-01-01
795573   2017-02-01
798214   2017-03-01
800855   2017-04-01
803496   2017-05-01
806137   2017-06-01
808778   2017-07-01
811419   2

In [36]:
#see the range of CPI index for Canada in 2018 and 2019 for any item
print("VALUE column for all types of products in the year 2018-2019", " is greater than 0: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 0, "greater_than"))
print("VALUE column for all types of products in the year 2018-2019", " is greater than 100: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 100, "greater_than"))
print("VALUE column for all types of products in the year 2018-2019", " is greater than 140: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 140, "greater_than"))
print("VALUE column for all types of products in the year 2018-2019", " is greater than 150: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 150, "greater_than"))
print("VALUE column for all types of products in the year 2018-2019", " is greater than 200: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 200, "greater_than"))
print("VALUE column for all types of products in the year 2018-2019", " is greater than 250: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 250, "greater_than"))
print("VALUE column for all types of products in the year 2018-2019", " is less than 500: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 500, "less_than"))
print("VALUE column for all types of products in the year 2018-2019", " is less than 800: ", test_unit_less_than_or_greater_than(df_Canada_CPI_Formatted, "VALUE", 800, "less_than"))


VALUE column for all types of products in the year 2018-2019  is greater than 0:  True
VALUE column for all types of products in the year 2018-2019  is greater than 100:  True
VALUE column for all types of products in the year 2018-2019  is greater than 140:  True
VALUE column for all types of products in the year 2018-2019  is greater than 150:  True
VALUE column for all types of products in the year 2018-2019  is greater than 200:  True
VALUE column for all types of products in the year 2018-2019  is greater than 250:  True
VALUE column for all types of products in the year 2018-2019  is less than 500:  True
VALUE column for all types of products in the year 2018-2019  is less than 800:  True


In [37]:
print("VALUE column for all type of products is between 15 and 260", test_unit_between(df_Canada_CPI_Formatted, "VALUE", 15, 260))

print(df_Canada_CPI_Formatted["VALUE"].describe())
print("Describe Education for Canadian Dataset: ", sep='/n')
print(df_Canada_CPI_Formatted[df_Canada_CPI_Formatted['Products and product groups']=="Education"]["VALUE"].describe())
print("Describe Energy for Canadian Dataset: ", sep='/n')
print(df_Canada_CPI_Formatted[df_Canada_CPI_Formatted['Products and product groups']=="Energy"]["VALUE"].describe())

VALUE column for all type of products is between 15 and 260 True
count    10266.000000
mean       133.178760
std         32.497847
min         17.200000
25%        113.338462
50%        135.476923
75%        151.400000
max        255.618182
Name: VALUE, dtype: float64
Describe Education for Canadian Dataset: 
count     38.000000
mean     158.809809
std        3.546890
min      153.272727
25%      157.252273
50%      157.413636
75%      161.956818
max      162.981818
Name: VALUE, dtype: float64
Describe Energy for Canadian Dataset: 
count     38.000000
mean     162.509109
std        7.559468
min      149.123077
25%      156.507692
50%      163.888462
75%      168.138462
max      174.592308
Name: VALUE, dtype: float64


In [38]:
Sales_Canada_Groceries = pd.read_csv('../data/raw/20100082.csv')



In [39]:
#It seems like the values for American dataset are much higher than the Canadian dataset. This is because the base year for the American dataset is 1984 while the base year for the Canadian dataset is 2002. This means that the Canadian dataset is indexed to 2002 = 100 while the American dataset is indexed to 1984 = 100. This means that the American dataset will have higher values than the Canadian dataset.
#It may be important to standardize the datasets when doing differences in differences for eventual comparison purposes but the numbers of CPI index appear to make sense. 

In [40]:
df_Canada_CPI_Formatted

,Products and product groups,REF_DATE,VALUE
0,Air transportation,2017-01-01,134.8
1,Air transportation,2017-02-01,135.2
2,Air transportation,2017-03-01,133.7
3,Air transportation,2017-04-01,134.7
4,Air transportation,2017-05-01,139.4
...,...,...,...
10261,Wooden furniture,2019-10-01,94.5
10262,Wooden furniture,2019-11-01,96.8
10263,Wooden furniture,2019-12-01,96.8
10264,Wooden furniture,2020-01-01,94.4


In [41]:
print(df_Canada_Number_Groceries.head())

df_Canada_Number_Groceries = manipulate_data(df_Canada_Number_Groceries, 'Canada')

  REF_DATE     GEO           DGUID  \
0  2017-01  Canada  2016A000011124   
1  2017-01  Canada  2016A000011124   
2  2017-01  Canada  2016A000011124   
3  2017-01  Canada  2016A000011124   
4  2017-01  Canada  2016A000011124   

  North American Industry Classification System (NAICS)  \
0                               Retail trade [44-45]      
1                               Retail trade [44-45]      
2                               Retail trade [44-45]      
3                               Retail trade [44-45]      
4              Motor vehicle and parts dealers [441]      

                     Sales          Adjustments      UOM  UOM_ID  \
0       Total retail sales           Unadjusted  Dollars      81   
1       Total retail sales  Seasonally adjusted  Dollars      81   
2  Retail e-commerce sales           Unadjusted  Dollars      81   
3  Retail e-commerce sales  Seasonally adjusted  Dollars      81   
4       Total retail sales           Unadjusted  Dollars      81   

  SCALA

In [42]:
df_Canada_Number_Groceries.head(50)

,Sales,North American Industry Classification System (NAICS),REF_DATE,VALUE
1,Total retail sales,Retail trade [44-45],2017-01-01,50417235.0
3,Retail e-commerce sales,Retail trade [44-45],2017-01-01,1236885.0
5,Total retail sales,Motor vehicle and parts dealers [441],2017-01-01,13561527.0
7,Total retail sales,Automobile dealers [4411],2017-01-01,11980440.0
9,Total retail sales,New car dealers [44111],2017-01-01,11020613.0
11,Total retail sales,Used car dealers [44112],2017-01-01,959827.0
13,Total retail sales,Other motor vehicle dealers [4412],2017-01-01,855975.0
15,Total retail sales,"Automotive parts, accessories and tire retaile...",2017-01-01,725113.0
17,Total retail sales,Building material and garden equipment and sup...,2017-01-01,2932069.0
19,Total retail sales,Food and beverage retailers [445],2017-01-01,10070575.0


In [43]:
save_data(df_USA_CPI_Formatted, '../data/processed/USA_CPI_Processed_2018_2019.csv')
save_data(df_Canada_CPI_Formatted, '../data/processed/Canada_CPI_Processed_2018_2019.csv')
save_data(df_Canada_Sales_Formatted, '../data/processed/Canada_Sales_Processed.csv')
save_data(df_USA_Sales, '../data/processed/USA_Sales_Processed.csv')
save_data(df_Canada_Number_Groceries, '../data/processed/Canada_Number_Groceries_Processed.csv')

In [44]:
df_USA_CPI_Formatted['REF_DATE'].tail()

43   2020-08-01
44   2020-09-01
45   2020-10-01
46   2020-11-01
47   2020-12-01
Name: REF_DATE, dtype: datetime64[ns]

In [45]:
df_sales = pd.read_csv('../data/raw/16100047.csv')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_32154/3005955346.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sales = pd.read_csv('../data/raw/16100047.csv')
